In [ ]:
!nvidia-smi -L

GPU 0: Tesla P100-PCIE-16GB (UUID: GPU-1e530248-c508-d846-76e2-7c8a025f6eae)


In [ ]:
data_path = "'/content/drive/MyDrive/Colab Notebooks/AiffelThon/jemu/season.zip'"

In [ ]:
%cd /content
!unzip -q {data_path}

/content


In [ ]:
fall = '/content/F'
spring = '/content/S'

In [ ]:
import os

%cd /content/F
for idx, img in enumerate(os.listdir(fall)):
  idx = str(idx).zfill(5)
  os.rename(img, idx + '.jpeg')

/content/F


In [ ]:
%cd /content/S
for idx, img in enumerate(os.listdir(spring)):
  idx = str(idx).zfill(5)
  os.rename(img, idx + '.jpeg')

/content/S


In [ ]:
len(os.listdir('/content/F')), len(os.listdir('/content/S'))

(2380, 2230)

In [ ]:
%cd /content
!mkdir dataset
!mv F dataset
!mv S dataset

/content


In [ ]:
!cp -r {data_path} /content
!mv /content/season/F.W /content/season/0
!mv /content/season/S.S /content/season/1

mv: cannot move '/content/season/F.W' to '/content/season/0': Operation not permitted
mv: cannot move '/content/season/S.S' to '/content/season/1': Operation not permitted


In [ ]:
import torch
import torch.nn as nn
from torchvision.datasets import ImageFolder
import torchvision.transforms as T
from torch.utils.data.sampler import SubsetRandomSampler

import numpy as np

In [ ]:
import os
import shutil
import glob
import math
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler

import torchvision
import torchvision.transforms as T
from torchsummary import summary

In [ ]:
path = '/content/dataset'
transforms = T.Compose([T.Resize((224, 224)), T.ToTensor()])
train_data = ImageFolder(path, transforms)

In [ ]:
# {'F': 0, 'S': 1}
train_data.class_to_idx

{'F': 0, 'S': 1}

In [ ]:
save_path = '/content/drive/MyDrive/Colab Notebooks/AiffelThon/project/season/model_weights.pth'

In [ ]:
valid_size = 0.2
batch_size = 128
train_size = len(train_data)

num_train = len(train_data)
indices = list(range(num_train))
np.random.shuffle(indices)
split = int(np.floor(valid_size * num_train))
train_idx, valid_idx = indices[split:], indices[:split]

# trainning, validation batch를 얻기 위한 sampler정의
train_sampler = SubsetRandomSampler(train_idx)
valid_sampler = SubsetRandomSampler(valid_idx)

# load training data in batches
train_loader = torch.utils.data.DataLoader(train_data,
                                            batch_size=batch_size,
                                            sampler=train_sampler,
                                            num_workers=2)

# load validation data in batches
valid_loader = torch.utils.data.DataLoader(train_data,
                                            batch_size=batch_size,
                                            sampler=valid_sampler,
                                            num_workers=2)

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = torch.hub.load('pytorch/vision:v0.10.0', 'resnet50', pretrained=True)
model.fc = nn.Linear(2048, 2)
model = model.to(device)

Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.10.0


In [ ]:
def train(dataloader, model, loss_fn, optimizer):
  size = train_size
  size = int(size*0.8)

  model.train()
  for batch, (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    pred = model(X)
    loss = loss_fn(pred, y)

    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    gap = math.ceil(size//batch_size / 5 )
    if batch % gap == 0:
      loss, current = loss.item(), batch * len(X)
      print(f'loss : {loss:>7f} [{current/size*100:>0.2f}%]; current: {current:4d}/{size}')

In [60]:
def test(dataloader, model, loss_fn):
  size = int(train_size*0.2)
  num_batches = len(dataloader)

  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
  return test_loss

In [ ]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [ ]:
epochs = 20

min_loss = np.Inf
for t in range(epochs):
  print(f'epoch : {t+1} \n-------------------')
  train(train_loader, model, loss_fn, optimizer)
  val_loss = test(valid_loader, model, loss_fn)

  if min_loss > val_loss:
    min_loss = val_loss
    print("Model is saved")
    torch.save(model.state_dict(), save_path)
print('Done')

# Test

In [65]:
def test(dataloader, model, loss_fn):
  size = int(train_size*0.2)
  num_batches = len(dataloader)

  model.eval()
  test_loss, correct = 0, 0
  with torch.no_grad():
    for X, y in dataloader:
      X, y = X.to(device), y.to(device)
      pred = model(X)
      test_loss += loss_fn(pred, y).item()
      correct += (pred.argmax(1) == y).type(torch.float).sum().item()

  test_loss /= num_batches
  correct /= size
  print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
  return test_loss

In [66]:
epochs = 1

min_loss = np.Inf
for t in range(epochs):
  print(f'epoch : {t+1} \n-------------------')
  val_loss = test(valid_loader, model, loss_fn)

  if min_loss > val_loss:
    min_loss = val_loss
    print("Model is saved")
    torch.save(model.state_dict(), save_path)
print('Done')

epoch : 1 
-------------------
Test Error: 
 Accuracy: 100.0%, Avg loss: 0.000531 

Model is saved
Done
